In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
data = pd.read_csv("C:/temp/destinations_LP_crawler_Ex5 .csv",encoding='latin-1')


In [3]:
type(data)

pandas.core.frame.DataFrame

In [4]:
data['city']

0       Antalya
1        Athens
2     Ayia-napa
3       Bangkok
4     Barcelona
        ...    
71       Venice
72       Vienna
73       Warsaw
74    Zakynthos
75       Zurich
Name: city, Length: 76, dtype: object

In [5]:
api_key = my_api'
def GETNAME(row):
    city=row['city']
    try:
            urlgeocode="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (city,api_key)
            response_urlgeocode = requests.get(urlgeocode)
            response_data_geocode = response_urlgeocode.json()
            cuntrydic=response_data_geocode['results'][0]['address_components']
            for i in cuntrydic:
                if [i][0]['types'][0]=='country':
                    return([i][0]['long_name'])
                                
    except:
            try:
                city=city.split('-')[0]
                urlgeocode="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (city,api_key)
                response_urlgeocode = requests.get(urlgeocode)
                response_data_geocode = response_urlgeocode.json()
                cuntrydic=response_data_geocode['results'][0]['address_components']
                for i in cuntrydic:
                       if [i][0]['types'][0]=='country':
                             return([i][0]['long_name'])
            except:
                    try:
                        city=row['city_LP']
                        urlgeocode="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (city,api_key)
                        response_urlgeocode = requests.get(urlgeocode)
                        response_data_geocode = response_urlgeocode.json()
                        cuntrydic=response_data_geocode['results'][0]['address_components']
                        for i in cuntrydic:
                            if [i][0]['types'][0]=='country':
                                return([i][0]['long_name'])
                    except:   
                        return np.NaN

data['Country']=data.apply(lambda row:GETNAME(row),axis=1)

In [6]:
data['Country'].unique()


array(['Turkey', 'Greece', 'Cyprus', 'Thailand', 'Spain', 'Switzerland',
       'Georgia', 'China', 'Germany', 'United States', 'Slovakia',
       'Belgium', 'Romania', 'Hungary', 'Bulgaria', 'Mexico', 'Italy',
       'Poland', 'India', 'Ireland', 'Croatia', 'Austria', 'Ukraine',
       'Portugal', 'Slovenia', 'Philippines', 'France', 'Russia',
       'Taiwan', 'Canada'], dtype=object)

In [7]:
data

,city,city_LP,Country
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
...,...,...,...
71,Venice,Venice,Italy
72,Vienna,Vienna,Austria
73,Warsaw,Warsaw,Poland
74,Zakynthos,Zakynthos,Greece


In [8]:
#we have none?
data[data['Country'].isnull()]

#data[data['Country']=='Poland']


,city,city_LP,Country


In [9]:
#data1=data[data['city']=='Venice']
data1=data.iloc[1:10]

In [10]:
data1

,city,city_LP,Country
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
5,Basel,northern-switzerland/basel,Switzerland
6,Batumi,adjara/batumi,Georgia
7,Beijing,Beijing,China
8,Berlin,Berlin,Germany
9,Boston,Boston,United States


In [11]:
def GetDescription(row):
    try:
        url="https://www.lonelyplanet.com/"+row['Country']+"/"+row['city_LP']
        html = requests.get(url)
        soup = BeautifulSoup(html.content,'html.parser')
        diteils_list=list()
        diteils=soup.find('div',class_='readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg')
        for dit in diteils:
            if dit.name=='p':
                diteils_list.append(dit.text)
            elif dit.name=='h2':
                diteils_list.append(dit.text+':')
    #convent the list to be a string
        diteils_list_str=' '.join([str(item) for item in diteils_list])
        return(diteils_list_str)
    except:
             return np.NaN

data['Description']=data.apply(lambda row:GetDescription(row),axis=1)


In [12]:
#do we have null?
#WE can see we have null when the cuntry name is United States so i change to USA and apply again the function
data[data['Description'].isnull()]


,city,city_LP,Country,Description
9,Boston,Boston,United States,NaN
39,Las-Vegas,Las-Vegas,United States,NaN
43,Los Angeles,Los-Angeles,United States,NaN
48,Miami,Miami,United States,NaN
62,San-Francisco,San-Francisco,United States,NaN


In [13]:
data['Country']=data['Country'].apply(lambda x: 'USA' if x == 'United States' else x)

In [14]:
data[data['Country']=='USA']

,city,city_LP,Country,Description
9,Boston,Boston,USA,NaN
39,Las-Vegas,Las-Vegas,USA,NaN
43,Los Angeles,Los-Angeles,USA,NaN
48,Miami,Miami,USA,NaN
62,San-Francisco,San-Francisco,USA,NaN


In [15]:
data['Description']=data.apply(lambda row:GetDescription(row),axis=1)

In [17]:
data[data['Country']=='USA']

,city,city_LP,Country,Description
9,Boston,Boston,USA,Boston's history recalls revolution and transf...
39,Las-Vegas,Las-Vegas,USA,"An oasis of indulgence in the desert, Vegas' s..."
43,Los Angeles,Los-Angeles,USA,"Ruggedly good looking, deeply creative, with a..."
48,Miami,Miami,USA,Beautiful beaches and art deco delights are ju...
62,San-Francisco,San-Francisco,USA,"Other cities may surprise you, but in San Fran..."


In [18]:
data[data['Description'].isnull()]


,city,city_LP,Country,Description


In [19]:
LP_destinations=data[['city','Country','Description']]
LP_destinations

,city,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
71,Venice,Italy,Imagine the audacity of building a city of mar...
72,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
73,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
74,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [20]:
LP_destinations.iloc[0,2]

"Once seen simply as the gateway to the Turkish Riviera, Antalya today is very much a destination in its own right. Situated right on the Gulf of Antalya (Antalya Körfezi), the largest city on Turkey's western Mediterranean coastline is both classically beautiful and stylishly modern. At its core is the wonderfully preserved old-city district of Kaleiçi (literally 'within the castle'), which offers atmospheric accommodation in the finely restored Ottoman houses on its winding lanes. The old city wraps around a splendid Roman-era harbour with clifftop views of hazy-blue mountain silhouettes that are worth raising a toast to. Just outside of the central city are two beaches and one of Turkey's finest museums."

In [21]:
LP_destinations.to_csv("C:/temp/LP_destinations.csv",index=False)